In [1]:
# 1. colab & google drive를 연동
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# 2. 필요한 라이브러리를 불러옴
!pip install konlpy

In [3]:
# 2. 필요한 라이브러리를 불러옴
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

In [5]:
# 3. 데이터를 전처리합니다.
train = pd.read_csv('/content/gdrive/MyDrive/Colab/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/gdrive/MyDrive/Colab/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/gdrive/MyDrive/Colab/sample_submission.csv', encoding = 'utf-8')

In [6]:
# 3. 데이터를 전처리합니다.
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [7]:
# 3. 데이터를 전처리합니다.
okt = Okt()

In [ ]:
# 3. 데이터를 전처리합니다.
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

 22%|██▏       | 8995/39992 [06:06<35:33, 14.53it/s]

In [ ]:
# 3. 데이터를 전처리합니다.
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [ ]:
# 3. 데이터를 전처리합니다.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
# 3. 데이터를 전처리합니다.
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# 3. 데이터를 전처리합니다.
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
# 3. 데이터를 전처리합니다.
y_train = to_categorical(train['category'])

In [ ]:
# 4. 모델을 생성 및 훈련합니다.
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 42s 108ms/step - loss: 0.5204 - acc: 0.7988
Epoch 2/15
313/313 [==============================] - 33s 106ms/step - loss: 0.3223 - acc: 0.8864
Epoch 3/15
313/313 [==============================] - 34s 107ms/step - loss: 0.2359 - acc: 0.9178
Epoch 4/15
313/313 [==============================] - 34s 108ms/step - loss: 0.1864 - acc: 0.9346
Epoch 5/15
313/313 [==============================] - 33s 107ms/step - loss: 0.2064 - acc: 0.9256
Epoch 6/15
313/313 [==============================] - 33s 106ms/step - loss: 0.2185 - acc: 0.9231
Epoch 7/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1493 - acc: 0.9473
Epoch 8/15
313/313 [==============================] - 34s 107ms/step - loss: 0.1556 - acc: 0.9463
Epoch 9/15
313/313 [==============================] - 33s 107ms/step - loss: 0.1076 - acc: 0.9612
Epoch 10/15
313/313 [==============================] - 33s 107ms/step - loss: 0.0873 - acc: 0.9689
Epoch 11/15
313/313

In [ ]:
# 5. 훈련된 모델로 예측, test에 대한 submission 파일을 생성합니다.
model.save('/content/gdrive/MyDrive/Colab/model')

#y_pred = model.predict_classes(X_test)
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab/bluehouse/model/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab/bluehouse/model/assets


In [ ]:
# 5. 훈련된 모델로 예측, test에 대한 submission 파일을 생성합니다.
import datetime

sample_submission['category'] = classes_x
sample_submission.to_csv('/content/gdrive/MyDrive/Colab/result-test.csv', encoding='utf-8', index=False)

In [ ]:
# 6. input data를 전처리합니다.
input['data'] = input['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
# 6. input data를 전처리합니다.
input = pd.read_csv('/content/gdrive/MyDrive/Colab/input.csv', encoding = 'euc-kr')

In [ ]:
# 6. input data를 전처리합니다.
X_input = []
for sentence in input['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_input.append(temp_X)

In [ ]:
# 6. input data를 전처리합니다.
X_input = tokenizer.texts_to_sequences(X_input)

In [ ]:
# 6. input data를 전처리합니다.
X_input = pad_sequences(X_input, maxlen = max_len)

In [ ]:
# 5. 훈련된 모델로 input에 대해 예측, submission 파일을 생성합니다.
predict_x=model.predict(X_input) 
classes_x=np.argmax(predict_x,axis=1)

sample_submission['category'] = classes_x
sample_submission.to_csv('/content/gdrive/MyDrive/Colab/result-input.csv', encoding='utf-8', index=False)